In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

data = pd.read_excel("/content/Main Data Set.xlsx")
x = data.iloc[:, :-2]
y = data.iloc[:, -1]

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [3]:
# Generator
latent_dim = 100  # Dimensionality of the generator input
generator_input_shape = (latent_dim,)
generator_inputs = Input(shape=generator_input_shape)
generator_dense = Dense(64, activation='relu')(generator_inputs)
generator_output = Dense(x_scaled.shape[1], activation='tanh')(generator_dense)

generator = Model(inputs=generator_inputs, outputs=generator_output)

In [4]:
# Discriminator
data_dim = x_scaled.shape[1]
discriminator_input_shape = (data_dim,)
discriminator_inputs = Input(shape=discriminator_input_shape)
discriminator_dense = Dense(64, activation='relu')(discriminator_inputs)
discriminator_output = Dense(1, activation='sigmoid')(discriminator_dense)

discriminator = Model(inputs=discriminator_inputs, outputs=discriminator_output)


In [5]:
# Combined model (Generator + Discriminator)
combined_input = Input(shape=generator_input_shape)
combined_output = discriminator(generator(combined_input))

combined = Model(inputs=combined_input, outputs=combined_output)


In [6]:
# Compile the models
generator.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
discriminator.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
combined.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')


In [7]:
# Training loop
num_epochs = 30
batch_size = 32


In [8]:
for epoch in range(num_epochs):
    # Train the discriminator
    real_data = x_scaled[np.random.randint(0, x_scaled.shape[0], batch_size)]
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_data = generator.predict(noise)

    discriminator_loss_real = discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
    discriminator_loss_fake = discriminator.train_on_batch(fake_data, np.zeros((batch_size, 1)))
    discriminator_loss = 0.5 * (discriminator_loss_real + discriminator_loss_fake)

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    generator_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

    # Print the losses
    print(f"Epoch: {epoch+1}/{num_epochs}, Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}")


1/1 [==============================] - 3s 3s/step
Epoch: 1/30, Discriminator Loss: 0.7150730788707733, Generator Loss: 0.7100780010223389
1/1 [==============================] - 0s 19ms/step
Epoch: 2/30, Discriminator Loss: 0.7166656255722046, Generator Loss: 0.6795130968093872
1/1 [==============================] - 0s 18ms/step
Epoch: 3/30, Discriminator Loss: 0.689085066318512, Generator Loss: 0.6455501317977905
1/1 [==============================] - 0s 16ms/step
Epoch: 4/30, Discriminator Loss: 0.6958202719688416, Generator Loss: 0.5862948894500732
1/1 [==============================] - 0s 18ms/step
Epoch: 5/30, Discriminator Loss: 0.6363355219364166, Generator Loss: 0.5694261789321899
1/1 [==============================] - 0s 20ms/step
Epoch: 6/30, Discriminator Loss: 0.6388570666313171, Generator Loss: 0.546149492263794
1/1 [==============================] - 0s 17ms/step
Epoch: 7/30, Discriminator Loss: 0.667407363653183, Generator Loss: 0.5012349486351013
1/1 [====================

In [9]:
# Generate samples from the generator
num_samples = 10
noise = np.random.normal(0, 1, (num_samples, latent_dim))
generated_samples = generator.predict(noise)

# Convert generated samples back to original scale
generated_samples = scaler.inverse_transform(generated_samples)

1/1 [==============================] - 0s 42ms/step


In [10]:
# Print generated samples
print("Generated Samples:")
print(generated_samples)

Generated Samples:
[[-4.26313416e+02  6.36648827e+01 -1.49565134e+01 -6.99190903e+01
   2.84823456e+01 -4.87243042e+01 -8.10649490e+01  9.88577127e-01
   1.31514997e+01 -5.72720451e+01 -6.19182549e+01 -2.34318581e+01
  -4.90401230e+01 -5.69962196e+01 -8.19638920e+00  5.26297140e+00
   6.36754274e-01 -1.98074627e+01 -2.79168034e+01 -1.62242401e+00
  -1.58375759e+01 -3.39308205e+01 -4.92911758e+01  1.63362179e+01
   1.87164021e+01 -2.35190868e+01  1.86038342e+01  2.04668121e+01
  -3.25481873e+01 -1.01147690e+01 -3.95093651e+01 -7.24769258e+00
   1.05133152e+01 -7.81134892e+00 -1.21058273e+01 -3.02799702e+00
   6.25444078e+00  3.34824562e+00 -2.33560505e+01]
 [-1.85441315e+02  8.76640320e+01 -3.63428092e+00 -8.11708908e+01
   2.02120018e+00 -5.97188568e+01 -1.02377129e+02 -1.69070077e+00
   1.68029938e+01 -2.28826656e+01 -7.43548889e+01  8.38896275e+00
  -6.12182274e+01 -5.32765236e+01 -5.85506897e+01 -4.11669769e+01
  -1.76772842e+01 -5.61194754e+00  5.67931700e+00  9.82296944e+00
   1.2

In [15]:
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_encoded, test_size=0.2, random_state=42)
input_shape = (x_train.shape[1],)

class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)))

inputs = Input(shape=input_shape)
x = Dense(256, activation='relu')(inputs)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='tanh')(x)  # Change activation function to 'tanh'
x = Dropout(0.5)(x)
x = Dense(32, activation='sigmoid')(x)  # Change activation function to 'sigmoid'
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=15, callbacks=[early_stopping])



Epoch 1/15
75/75 [==============================] - 3s 6ms/step - loss: 0.6766 - accuracy: 0.5804 - val_loss: 0.3087 - val_accuracy: 0.9783
Epoch 2/15
75/75 [==============================] - 0s 4ms/step - loss: 0.2459 - accuracy: 0.9283 - val_loss: 0.0669 - val_accuracy: 0.9867
Epoch 3/15
75/75 [==============================] - 0s 4ms/step - loss: 0.1107 - accuracy: 0.9733 - val_loss: 0.0387 - val_accuracy: 0.9950
Epoch 4/15
75/75 [==============================] - 0s 4ms/step - loss: 0.0982 - accuracy: 0.9750 - val_loss: 0.0315 - val_accuracy: 0.9950
Epoch 5/15
75/75 [==============================] - 0s 5ms/step - loss: 0.0847 - accuracy: 0.9779 - val_loss: 0.0267 - val_accuracy: 0.9950
Epoch 6/15
75/75 [==============================] - 0s 4ms/step - loss: 0.0795 - accuracy: 0.9804 - val_loss: 0.0293 - val_accuracy: 0.9950
Epoch 7/15
75/75 [==============================] - 0s 4ms/step - loss: 0.0719 - accuracy: 0.9833 - val_loss: 0.0223 - val_accuracy: 0.9950
Epoch 8/15
75/75 [==

In [16]:
# Evaluate the model
_, train_accuracy = model.evaluate(x_train, y_train)
_, test_accuracy = model.evaluate(x_test, y_test)


19/19 [==============================] - 0s 2ms/step - loss: 0.0204 - accuracy: 0.9950


In [17]:
print('Training Accuracy:', train_accuracy)
print('Testing Accuracy:', test_accuracy)

Training Accuracy: 0.9954166412353516
Testing Accuracy: 0.9950000047683716


In [18]:
# Make predictions
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)
# Generate classification report
report = classification_report(y_test, y_pred)
print(report)

19/19 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       313
           1       1.00      0.99      0.99       287

    accuracy                           0.99       600
   macro avg       1.00      0.99      0.99       600
weighted avg       1.00      0.99      0.99       600



In [19]:
# Use the age estimation model to predict ages for the generated samples
generated_samples_pred = model.predict(generated_samples)

1/1 [==============================] - 0s 81ms/step


In [22]:
generated_samples_pred =(generated_samples_pred > 0.5).astype(int)
generated_samples_pred 

array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])